In [18]:
# Regular python libraries
from ipywidgets import interact
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plot
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Standar bokeh libraries
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import LinearAxis, Range1d  # Handle two y axis
from bokeh.models import CheckboxGroup, CustomJS
from bokeh.layouts import row, column, widgetbox
#from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [3]:
# Read data (from file or DB)
df = pd.read_csv('../Dataset/train/00001/acceleration.csv',# skiprows=5,
    sep=',')
df.head()


,t,x,y,z,Kitchen_AP,Lounge_AP,Upstairs_AP,Study_AP
0,0.017856,0.944,-0.280,0.152,-93.0,-95.0,-79.0,NaN
1,0.067904,0.944,-0.292,0.156,-93.0,-95.0,-79.0,NaN
2,0.117952,0.946,-0.286,0.156,-93.0,-95.0,-79.0,NaN
3,0.168000,0.942,-0.288,0.160,-93.0,-95.0,-79.0,NaN
4,0.217856,0.946,-0.286,0.158,-90.0,-89.0,-79.0,NaN


In [136]:
dfl = pd.read_csv('../Dataset/train/00001/location_1.csv',# skiprows=5,
    sep=',')
dfl['end'] = dfl['end']+.01
dfl.head()

,start,end,name,index
0,45.930,48.100,hall,3
1,48.090,427.258,living,5
2,427.248,434.258,hall,3
3,434.248,714.196,kitchen,4
4,714.186,720.196,hall,3


In [8]:
dfa = df.copy()
dfa['t'] = pd.to_timedelta(dfa['t'],unit='s')
#dfa.head()
dfa = dfa.set_index(pd.TimedeltaIndex(dfa['t']))
#dfb.head()
del dfa['t']
dfa.head()

,x,y,z,Kitchen_AP,Lounge_AP,Upstairs_AP,Study_AP
00:00:00.017856,0.944,-0.280,0.152,-93.0,-95.0,-79.0,NaN
00:00:00.067904,0.944,-0.292,0.156,-93.0,-95.0,-79.0,NaN
00:00:00.117952,0.946,-0.286,0.156,-93.0,-95.0,-79.0,NaN
00:00:00.168000,0.942,-0.288,0.160,-93.0,-95.0,-79.0,NaN
00:00:00.217856,0.946,-0.286,0.158,-90.0,-89.0,-79.0,NaN


In [152]:
dfla = pd.concat([dfl['start'].dropna(),dfl['end'].dropna()])  
dflb = dfla.to_frame() 
dflb.columns = ['t'] 
dflb['index'] = dfl['index'][dflb.index.values]
dflb['name'] = dfl['name'][dflb.index.values]
dflb = dflb.sort().reset_index()
del dflb['level_0']
dflb['t'] = pd.to_timedelta(dflb['t'],unit='s')
#dflb.head()
dflb = dflb.set_index(pd.TimedeltaIndex(dflb['t']))
#dflb.head()
del dflb['t']
dflb = dflb.sort()
dflb.head()

,index,name
00:00:45.930000,3,hall
00:00:48.090000,5,living
00:00:48.100000,3,hall
00:07:07.248000,3,hall
00:07:07.258000,5,living


In [154]:
dflb['map_index'] = dflb['name'].replace(['bath', 'bed1', 'bed2', 'hall', 'kitchen', 'living', 'stairs', 'study', 'toilet'],[3, 2, 4, 1, 3, 1, 4, 2, 4])
dflb

,index,name,map_index
00:00:45.930000,3,hall,1
00:00:48.090000,5,living,1
00:00:48.100000,3,hall,1
00:07:07.248000,3,hall,1
00:07:07.258000,5,living,1
00:07:14.248000,4,kitchen,3
00:07:14.258000,3,hall,1
00:11:54.186000,3,hall,1
00:11:54.196000,4,kitchen,3
00:12:00.186000,5,living,1


In [46]:
dfb = dfa[['Kitchen_AP','Lounge_AP','Upstairs_AP','Study_AP']]
dfb.head()

,Kitchen_AP,Lounge_AP,Upstairs_AP,Study_AP
00:00:00.017856,-93.0,-95.0,-79.0,NaN
00:00:00.067904,-93.0,-95.0,-79.0,NaN
00:00:00.117952,-93.0,-95.0,-79.0,NaN
00:00:00.168000,-93.0,-95.0,-79.0,NaN
00:00:00.217856,-90.0,-89.0,-79.0,NaN


In [47]:
dfb['plocation'] = dfb.idxmax(axis=1,skipna=True)
dfb.head()

,Kitchen_AP,Lounge_AP,Upstairs_AP,Study_AP,plocation
00:00:00.017856,-93.0,-95.0,-79.0,NaN,Upstairs_AP
00:00:00.067904,-93.0,-95.0,-79.0,NaN,Upstairs_AP
00:00:00.117952,-93.0,-95.0,-79.0,NaN,Upstairs_AP
00:00:00.168000,-93.0,-95.0,-79.0,NaN,Upstairs_AP
00:00:00.217856,-90.0,-89.0,-79.0,NaN,Upstairs_AP


In [155]:
dfb['ilocation']=dfb['plocation'].replace(['Kitchen_AP', 'Lounge_AP', 'Upstairs_AP', 'Study_AP'],[3,1,4,2])
dfb.head()

,Kitchen_AP,Lounge_AP,Upstairs_AP,Study_AP,plocation,ilocation
00:00:00.017856,-93.0,-95.0,-79.0,NaN,Upstairs_AP,4
00:00:00.067904,-93.0,-95.0,-79.0,NaN,Upstairs_AP,4
00:00:00.117952,-93.0,-95.0,-79.0,NaN,Upstairs_AP,4
00:00:00.168000,-93.0,-95.0,-79.0,NaN,Upstairs_AP,4
00:00:00.217856,-90.0,-89.0,-79.0,NaN,Upstairs_AP,4


In [53]:
#dfb['ilocation'].plot()

In [156]:
f = figure(title="Just another try", plot_height=300, plot_width=900, x_axis_type="datetime")
plocation = f.circle(dfb.index,dfb['ilocation'], size=4, legend='plocation') 
location = f.line(dflb.index,dflb['map_index'], color = '#008744',line_width=2, legend='location') 

In [157]:
plocation.visible = True
location.visible = True
show(f)

In [ ]:
Continue with slicing to check mapping, something rare from ~7.5 to 12 m. Check transitions 

In [4]:
def sampling_data(timeunit, sampleps, from_time, to_time):
    freq = str(int(1000/sampleps))+'L'
    dfb = dfa.resample(freq).mean()
    dfc = dfb[dfb.index.slice_indexer(str(int(from_time))+timeunit,str(to_time-0.001)+timeunit,1)]
    return dfc

In [5]:
sampleps = 1
timeunit = 'M'
from_time = 0
to_time = 5
dfc = sampling_data(timeunit, sampleps, from_time, to_time)
#dfc.head()

x_index = dfc.index
y = dfc['x']

In [6]:
acc_dim = ['x','y','z']
dim_color = {'x':"#008744",'y':"#0057e7",'z':"#d62d20"}
f = figure(title="Just another try", plot_height=700, plot_width=900, x_axis_type="datetime")

# Setting the second y axis range name and range
f.extra_y_ranges = {"AP": Range1d(start=-4, end=2)}

# Adding the second axis to the plot.  
#f.add_layout(LinearAxis(y_range_name="AP"), 'right')
f.add_layout(LinearAxis(y_range_name="AP", axis_label='Acc Range'), 'right')

In [7]:
#def plot_dim(acc_dim,dim_color):
#    for dim in acc_dim:
#        var = "acc_"+dim
#        exec(var + "= f.line(dfc.index,dfc[dim], color=dim_color[dim], line_width=1, y_range_name='AP') ")
acc_x = f.line(dfc.index,dfc['x'], color=dim_color['x'], line_width=1, y_range_name='AP', legend='X') 
acc_y = f.line(dfc.index,dfc['y'], color=dim_color['y'], line_width=1, y_range_name='AP', legend='Y') 
acc_z = f.line(dfc.index,dfc['z'], color=dim_color['z'], line_width=1, y_range_name='AP', legend='Z') 

In [8]:
#plot_dim(acc_dim, dim_color)

In [9]:
aps = ['Kitchen_AP', 'Lounge_AP', 'Upstairs_AP', 'Study_AP']
ap_color = {'Kitchen_AP':"#008744",'Lounge_AP':"#0057e7",'Upstairs_AP':"#d62d20", 'Study_AP':"#ffa700"}

In [10]:
#def plot_ap(aps, ap_color):
#    for ap in aps:
#        var = "acc_"+ap
#        exec(var + "= f.circle(dfc.index,dfc[ap], size=4, color=ap_color[ap], alpha=0.2, legend=ap)")
acc_Kitchen_AP = f.circle(dfc.index,dfc['Kitchen_AP'], size=4, color=ap_color['Kitchen_AP'], alpha=0.2, legend='Kitchen_AP')
acc_Lounge_AP = f.circle(dfc.index,dfc['Lounge_AP'], size=4, color=ap_color['Lounge_AP'], alpha=0.2, legend='Lounge_AP')
acc_Upstairs_AP = f.circle(dfc.index,dfc['Upstairs_AP'], size=4, color=ap_color['Upstairs_AP'], alpha=0.2, legend='Upstairs_AP')
acc_Study_AP = f.circle(dfc.index,dfc['Study_AP'], size=4, color=ap_color['Study_AP'], alpha=0.2, legend='Study_AP')

In [11]:
#plot_ap(aps,ap_color)